In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import DataLoader, TensorDataset

In [5]:
class LightningLTSM(L.LightningModule):
    def __init__(self):
        super().__init__()

        # input size --> number of features/variables
        # hidden size --> number of output values we want (prediction for day 5)
        self.lstm = nn.LSTM(input_size=1, hidden_size=1)

    def forward(self, input):
        input_trans = input.transpose(0, 1)
        input_trans = input_trans.unsqueeze(-1)
        # lstm_out --> will contain each short term memory from each LSTM unit that we unrolled (in this case, it has 4 values) 
        lstm_out, temp = self.lstm(input_trans)
        prediction = lstm_out[-1]
        return prediction.squeeze()
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i)
        loss = F.mse_loss(output_i, label_i)
        self.log('train_loss', loss)

        if label_i.item() == 0:
            self.log('out_0', output_i)
        else:
            self.log('out_1', output_i)
            
        return loss

In [3]:
model = LightningLTSM()
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = tensor([0.3900])
Company B: Observed = 1, Predicted = tensor([0.3899])


In [6]:
model = LightningLTSM()
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs=300, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type | Params | Mode 
--------------------------------------
0 | lstm | LSTM | 16     | train
--------------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] 

C:\Users\11ukn\AppData\Local\Temp\ipykernel_19860\3331443930.py:23: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output_i, label_i)


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 104.55it/s, v_num=6]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 71.05it/s, v_num=6] 


In [7]:
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([[0., 0.5, 0.25, 1.]])).detach().item())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([[1., 0.5, 0.25, 1.]])).detach().item())


Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = 5.5710966989863664e-05
Company B: Observed = 1, Predicted = 0.9801440238952637
